<a href="https://colab.research.google.com/github/husthorng/Backpropagation_NN/blob/main/digitalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

import time
from datetime import datetime

# 狀態： [溫度觸發(>=28), 溫度 >=10秒, 光敏感]
lst = [0, 0, 0]

worksheet = gc.open_by_key("10VT8lpb7MIKEOMmKfqIKSh-yoNaYBsMx3mFdp8shjxU").worksheet("DHT")  #my_dht  1141husthorng
worksheet2 = gc.open_by_key("15sOASk3jTVCQNy6O-zwed0PXGIiE9JEB4PNc8H3XFcY").worksheet("input") #2025_DT hung 111husthorng


dt2 = None  # 記錄溫度開始達標的時間



while True:
    try:
        # ❗📌 讀取溫度
        temp_data = worksheet.get_values("C2")
        timestart = worksheet.get_values("A2")[0][0]
        temperature = float(temp_data[0][0])

        print(temperature)

        # 溫度上升 >= 28 → 觸發
        if temperature >= 28 and lst[0] == 0:
            lst[0] = 1
            dt2 = datetime.strptime(timestart, "%Y/%m/%d, %H:%M:%S")
            print("\n🔥 Temp Start:", timestart)

        # 溫度掉回 < 28 → 全部重置
        if temperature < 28 and lst[0] == 1:
            lst = [0, 0, 0]     # 全部清零
            dt2 = None
            print("\n🔄 Reset")

        # 溫度達標計時（有啟動 dt2 才計算）
        if lst[0] == 1 and dt2 is not None:
            timeend = worksheet.get_values("A3")[0][0]
            dt1 = datetime.strptime(timeend, "%Y/%m/%d, %H:%M:%S")
            diff = (dt1 - dt2).seconds
            print(f" ⏱ Duration: {diff} sec")
            print(diff)
            if diff >= 30 and diff < 86000:
                lst[1] = 1

        # 📌 讀取光敏感
        photo_data = worksheet.get_values("E2")
        Photosensitive = float(photo_data[0][0])

        # 📌 光敏感判斷
        lst[2] = 1 if Photosensitive <= 50 else 0

        print("📌狀態:", lst)

        # 📌 讀取 input 第一列（前3欄）比對是否相同
        rowi = worksheet2.row_values(1)
        row1 = np.array(rowi[:3], dtype=int).tolist()

        if row1 != lst:
            # 產生要寫入的資料：lst + ["", 時間]
            row_new = lst + ["", timestart]
            worksheet2.insert_row(row_new, 1)
            worksheet2.update([[timestart]], 'E1')
            print("💾 New Record Saved!")

    except Exception as e:
        print("⚠️ Get Value Error, Continue...", e)

    # 🕒 每 0.05 秒執行一次，不要造成 Google Sheet 堵塞
    time.sleep(0.5)


KeyboardInterrupt: 

In [ ]:

#將數值 數位化 已進行後續電腦運算
import numpy as np
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)


import time
from datetime import datetime

timestart=""
timeend=""
lst = [0, 0, 0]  # lst[0]: 溫度觸發, lst[1]: 溫度達10秒, lst[2]: 光敏感觸發

#  my_dht

worksheet2 = gc.open("2025_DT hung").worksheet("input")

worksheet = gc.open('my_dht').worksheet('DHT')
# 假設 lst 已初始化
#lst = [0, 0, 0]  # lst[0]: 溫度觸發, lst[1]: 溫度達10秒, lst[2]: 光敏感觸發
dt2 = None       # 記錄溫度觸發的起始時間

while True:
  # 📌 改成讀取 DHT 工作表


  # 讀取溫度
  temp_data = worksheet.get_values("C3")
  temperature = float(temp_data[0][0])
  #print("temperature:", temperature)
  timestart=worksheet.get_values("A3")[0][0]

  #print("tineend:", timeend)

  # 溫度上升到 >=28
  if temperature >= 28 and lst[0] == 0:
      lst[0] = 1
      #timestart=worksheet.get_values("A3")[0][0]
      dt2 = datetime.strptime(timestart, "%Y/%m/%d, %H:%M:%S")

      print("timestart:", timestart)



  # 降回 <29 → 重置
  if temperature < 20 and lst[0] == 1:
      lst[0] = 0
      lst[1] = 0
      dt2 = None
      #timestart=worksheet.get_values("A3")[0][0]
      print("Reset")

  # 溫度達標計時
  if lst[0] == 1 and dt2 is not None:
      timeend = worksheet.get_values("A3")[0][0]
      #timestart=worksheet.get_values("A3")[0][0]
      dt1 = datetime.strptime(timeend, "%Y/%m/%d, %H:%M:%S")
      print("timeend:", timeend)
      dt = (dt1 - dt2).seconds
      print("  dt1:", dt1)
      print("  dt2:", dt2)
      print("  Diff:", dt)


      if dt >= 28:  #經過10秒以上
          lst[1] = 1


  # 讀取光敏數值
  photo_data = worksheet.get_values("E3")
  Photosensitive = float(photo_data[0][0])

  # 光敏感觸發
  if Photosensitive <= 50:
      lst[2] = 1
      print("Photosensitive:", Photosensitive)
      #timestart=worksheet.get_values("A3")[0][0]
  else:
      lst[2] = 0
      #timestart=worksheet.get_values("A3")[0][0]

  # 印出目前狀態
  print(lst)
  print(timestart)
  rowi = worksheet2.row_values(1)

  rowi[:3]=np.array(rowi[:3], dtype=int)
  print(rowi[:3])

  lst1=lst[:3]
  lst1.append("")
  lst1.append(timestart)

  rowi = worksheet2.row_values(1)
  row1=rowi[:3]
  row1=np.array(row1, dtype=int)

  row1.tolist()==lst[:3]


  if row1.tolist()!=lst[:3]:
    worksheet2.insert_row(lst1, 1)
    worksheet2.update([[timestart]], 'E2')





# 每0.05秒檢查一次
time.sleep(50)

SpreadsheetNotFound: <Response [200]>